# Reading station weather data



In [9]:
# disable future warning
import warnings
warnings.simplefilter('ignore')

# other imports
from sqlalchemy import create_engine, inspect
from meteostat import Stations, Daily
import geopandas as gp
from shapely.geometry import Point
from datetime import date, datetime
from tqdm import tqdm

## Spacial Data
Each weather station has coordinates attached with it. To be able to merge it with the SOEP data set these coordinates need to be converted to some other format. We use the nuts format for now. 

In [10]:
# setup cache
Stations.cache_dir = "./prod/.meteostat/cache"

query = Stations()
query.region("DE")
stations = query.fetch()
stations.reset_index(inplace=True)

In [11]:
# remove unnecessary station data
cols = [
    'id',
    'latitude',
    'longitude',
    'elevation',
    'daily_start',
    'daily_end'
]
stations = stations[cols]
stations.rename({'id':'station_id'}, axis=1, inplace=True)

In [12]:
# read shape files
shape = [
    gp.read_file(f"./data/nuts5000/5000_NUTS{i}.shp").to_crs(epsg=4326)
    for i in range(1, 4)
]

# spacial join shape files. This is possible since nuts is hirarchical
nuts:gp.GeoDataFrame
nuts = shape[2].sjoin(shape[1], how="left", lsuffix='3', rsuffix='2', predicate="within")\
               .sjoin(shape[0], how="left", rsuffix='1', predicate="within")
nuts.rename({"NUTS_CODE":"NUTS_CODE_1",	"NUTS_NAME":"NUTS_NAME_1"}, inplace=True, axis=1)
nuts.drop("NUTS_LEVEL_2 NUTS_LEVEL_3 NUTS_LEVEL index_2 index_1".split(), inplace=True, axis=1)
nuts.sort_index(inplace=True)
nuts.head()

,NUTS_CODE_3,NUTS_NAME_3,geometry,NUTS_CODE_2,NUTS_NAME_2,NUTS_CODE_1,NUTS_NAME_1
0,DE111,"Stuttgart, Stadtkreis","POLYGON ((9.13453 48.85667, 9.14123 48.86183, ...",DE11,Stuttgart,DE1,Baden-Württemberg
1,DE112,Böblingen,"POLYGON ((8.96647 48.82979, 8.99217 48.83356, ...",DE11,Stuttgart,DE1,Baden-Württemberg
2,DE113,Esslingen,"POLYGON ((9.40974 48.53721, 9.39154 48.53014, ...",DE11,Stuttgart,DE1,Baden-Württemberg
3,DE114,Göppingen,"POLYGON ((9.91934 48.63977, 9.94731 48.63369, ...",DE11,Stuttgart,DE1,Baden-Württemberg
4,DE115,Ludwigsburg,"MULTIPOLYGON (((9.30157 48.95210, 9.31516 48.9...",DE11,Stuttgart,DE1,Baden-Württemberg


In [13]:
# create GeoDataFrame from stations
x, y = stations["longitude"], stations["latitude"]
stations["geometry"] = gp.GeoSeries(map(Point, zip(x, y)))
stations.drop(["longitude", "latitude"], axis=1, inplace=True)
stations = gp.GeoDataFrame(stations)

In [14]:
# spactial join with nuts data
stations:gp.GeoDataFrame
stations = stations.sjoin(nuts, how="inner", predicate="within")
stations.drop("index_right", axis=1, inplace=True)
stations.sort_index(inplace=True)
stations.head()

,station_id,elevation,daily_start,daily_end,geometry,NUTS_CODE_3,NUTS_NAME_3,NUTS_CODE_2,NUTS_NAME_2,NUTS_CODE_1,NUTS_NAME_1
1230,10015,4.0,1952-05-01,2022-11-26,POINT (7.90000 54.18330),DEF09,Pinneberg,DEF0,Schleswig-Holstein,DEF,Schleswig-Holstein
1231,10018,16.0,2009-02-24,2022-04-25,POINT (8.35000 54.91670),DEF07,Nordfriesland,DEF0,Schleswig-Holstein,DEF,Schleswig-Holstein
1232,10020,26.0,1931-01-01,2022-11-26,POINT (8.41670 55.01670),DEF07,Nordfriesland,DEF0,Schleswig-Holstein,DEF,Schleswig-Holstein
1233,10022,7.0,1973-01-01,2022-11-26,POINT (8.95000 54.80000),DEF07,Nordfriesland,DEF0,Schleswig-Holstein,DEF,Schleswig-Holstein
1234,10026,28.0,1891-01-01,1974-06-30,POINT (9.15000 54.51670),DEF07,Nordfriesland,DEF0,Schleswig-Holstein,DEF,Schleswig-Holstein


## Weather data
Now that we have all the spacial data the only thing left to do is to get the historical weather data

In [15]:
path = "prod/station.db"
con = create_engine("sqlite:///"+path, echo=False)

In [16]:
# setup cache
Daily.cache_dir = "./prod/.meteostat/cache"
Daily.max_age = 3000000 # approx 1 month cache time 
Daily.threads = 20

# start end endtime of SOEP panel
start = datetime(1985, 1, 1)
end = datetime.combine(date.today(), datetime.min.time())

# get stations that are already in db
insp = inspect(con)
tables = insp.get_table_names()

# fetch data
for _, station in tqdm(stations.iterrows(), total=stations.shape[0]):
    s_id = station["station_id"]
    if s_id in tables:
        continue
    daily = Daily(s_id, start=start, end=end)
    data = daily.fetch()
    data.to_sql(s_id, con)

100%|██████████| 1116/1116 [00:00<00:00, 17296.23it/s]
